In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [6]:
Data1=pd.read_csv("\\Users\\VISHESH\\OneDrive\\Desktop\\zepto\\Customers.csv")
Data2=pd.read_csv('\\Users\\VISHESH\OneDrive\\Desktop\\zepto\\Products.csv')
Data3=pd.read_csv("\\Users\\VISHESH\OneDrive\\Desktop\\zepto\\Transactions.csv")

<>:2: SyntaxWarning: invalid escape sequence '\O'
<>:3: SyntaxWarning: invalid escape sequence '\O'
<>:2: SyntaxWarning: invalid escape sequence '\O'
<>:3: SyntaxWarning: invalid escape sequence '\O'
C:\Users\VISHESH\AppData\Local\Temp\ipykernel_17996\2822957817.py:2: SyntaxWarning: invalid escape sequence '\O'
  Data2=pd.read_csv('\\Users\\VISHESH\OneDrive\\Desktop\\zepto\\Products.csv')
C:\Users\VISHESH\AppData\Local\Temp\ipykernel_17996\2822957817.py:3: SyntaxWarning: invalid escape sequence '\O'
  Data3=pd.read_csv("\\Users\\VISHESH\OneDrive\\Desktop\\zepto\\Transactions.csv")


In [7]:
merged_data = Data3.merge(Data1, on="CustomerID", how="left")
merged_data = merged_data.merge(Data2, on="ProductID", how="left")



In [8]:

customer_profiles = merged_data.groupby("CustomerID").agg(
    total_spending=("TotalValue", "sum"),
    transaction_count=("TransactionID", "count"),
    unique_categories=("Category", lambda x: x.nunique()),
    region=("Region", "first")
).reset_index()

label_encoder = LabelEncoder()
customer_profiles["region_encoded"] = label_encoder.fit_transform(customer_profiles["region"])

features = customer_profiles[["total_spending", "transaction_count", "unique_categories", "region_encoded"]]
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

similarity_matrix = cosine_similarity(features_scaled)

customer_ids = customer_profiles["CustomerID"].tolist()
lookalike_map = {}

for i in range(20):  
    similar_customers = sorted(
        enumerate(similarity_matrix[i]),
        key=lambda x: x[1],
        reverse=True
    )[1:4]  
    lookalike_map[customer_ids[i]] = [
        {"cust_id": customer_ids[j], "score": round(score, 3)}
        for j, score in similar_customers
    ]

# Save the lookalike map to Lookalike.csv
lookalike_df = pd.DataFrame.from_dict(lookalike_map, orient="index")
lookalike_df.to_csv("Vishesh_chavda_Lookalike.csv", index_label="CustomerID")